In [1]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from torch.utils.data import Dataset
import os
import pandas as pd
import requests
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef
from imblearn.metrics import specificity_score
import re
import warnings
import os
os.environ["WANDB_MODE"] = "disabled"
warnings.filterwarnings("ignore")

In [2]:
model_name = '../Rostlab/prot_bert'

In [3]:
class DeepLocDataset(Dataset):
    def __init__(self, split="train", tokenizer_name=model_name, max_length=80):
        self.datasetFolderPath = '../dataset/'
        self.trainFilePath = os.path.join(self.datasetFolderPath, 'CPPSet1-train.csv')
        self.testFilePath = os.path.join(self.datasetFolderPath, 'CPPSet1-test.csv')
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, do_lower_case=False)
        if split=="train":
            self.seqs, self.labels = self.load_dataset(self.trainFilePath)
        else:
            self.seqs, self.labels = self.load_dataset(self.testFilePath)
        self.max_length = max_length
        
    def load_dataset(self,path):
        df = pd.read_csv(path,names=['input','labels'],skiprows=1)
        seq = list(df['input'])
        seq = [' '.join(i) for i in seq]
        label = list(df['labels'])
        assert len(seq) == len(label)
        return seq, label

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        seq = " ".join("".join(self.seqs[idx].split()))
        seq = re.sub(r"[UZOB]", "X", seq)
        seq_ids = self.tokenizer(seq, truncation=True, padding='max_length', max_length=self.max_length)
        sample = {key: torch.tensor(val) for key, val in seq_ids.items()}
        sample['labels'] = torch.tensor(self.labels[idx])
        return sample

In [4]:
train_dataset = DeepLocDataset(split="train", tokenizer_name=model_name, max_length=80)
test_dataset = DeepLocDataset(split="test", tokenizer_name=model_name, max_length=80)

In [5]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    spec = specificity_score(labels, preds)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'specificity': spec,
        'f1': f1,
        'mcc' : matthews_corrcoef(labels, preds)
    }

In [6]:
def model_init():
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    #for param in model.parameters(): param.data = param.data.contiguous()
    return model

In [7]:
#pip install accelerate --upgrade -i http://pypi.douban.com/simple/

In [8]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    output_dir='../results',
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='../logs',
    logging_steps=2,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=30,
    fp16=True,
    fp16_opt_level="02",
    run_name="ProBert-BFD-MS",
    seed=3407)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics = compute_metrics)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../Rostlab/prot_bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../Rostlab/prot_bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Specificity,F1,Mcc
0,0.690600,0.689576,0.500000,0.500000,1.000000,0.000000,0.666667,0.000000
1,0.674900,0.619705,0.734783,0.654286,0.995652,0.473913,0.789655,0.550419
2,0.532500,0.516585,0.797826,0.714734,0.991304,0.604348,0.830601,0.645975
3,0.452900,0.316589,0.889130,0.827839,0.982609,0.795652,0.898608,0.792229
4,0.262400,0.253711,0.917391,0.880952,0.965217,0.869565,0.921162,0.838628
6,0.141900,0.198045,0.932609,0.942222,0.921739,0.943478,0.931868,0.865422
7,0.091800,0.199934,0.928261,0.930131,0.926087,0.930435,0.928105,0.856530
8,0.059300,0.265530,0.910870,0.912664,0.908696,0.913043,0.910675,0.821747
9,0.070600,0.263668,0.910870,0.888889,0.939130,0.882609,0.913319,0.823055
10,0.043800,0.252682,0.921739,0.911017,0.934783,0.908696,0.922747,0.843765


TrainOutput(global_step=60, training_loss=0.2085016158098976, metrics={'train_runtime': 685.2429, 'train_samples_per_second': 53.704, 'train_steps_per_second': 0.088, 'total_flos': 5238472338432000.0, 'train_loss': 0.2085016158098976, 'epoch': 15.652173913043478})

In [9]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.23550346493721008,
 'eval_accuracy': 0.9326086956521739,
 'eval_precision': 0.927038626609442,
 'eval_recall': 0.9391304347826087,
 'eval_specificity': 0.9260869565217391,
 'eval_f1': 0.9330453563714903,
 'eval_mcc': 0.865291001420344,
 'eval_runtime': 4.6575,
 'eval_samples_per_second': 98.766,
 'eval_steps_per_second': 6.227,
 'epoch': 15.652173913043478}

In [10]:
trainer.save_model('../model/ProtBert_CPPSet1.pt')

In [11]:
result = trainer.predict(test_dataset).predictions
golden = trainer.predict(test_dataset).label_ids
result = torch.argmax(torch.tensor(result), -1)

In [12]:
torch.save(result, '../model/CPPSet1_result1.pt')

In [13]:
torch.save(golden, '../model/golden.pt')